# Notebook 2 - Lyrics Preprocessing

In [1]:
# Needed Packages
from collections import Counter
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from string import punctuation
from better_profanity import profanity


In [2]:
# Read in compressed CSV file
raw_data_df = pd.read_csv("data/lyrics.csv.gz", compression = "gzip")

# Total Counts of Songs per Genre
print(raw_data_df["genre"].value_counts())

# Sample Table
raw_data_df.head()

rock       1000
rap        1000
r-b         984
pop         979
country     976
Name: genre, dtype: int64


,Unnamed: 0,artist,title,lyrics,genre
0,0,['Lil Nas X'],Old Town Road (Remix),"Oh, oh-oh\nOh\n\n[Chorus: Billy Ray Cyrus]\nYe...",country
1,1,['Taylor Swift'],All Too Well (10 Minute Version) (Taylor's Ver...,"I walked through the door with you, the air wa...",country
2,2,['Taylor Swift'],All Too Well (10 Minute Version) (Taylor's Ver...,"I walked through the door with you, the air wa...",country
3,3,['Lil Nas X'],Old Town Road,"Yeah, I'm gonna take my horse to the old town ...",country
4,4,['Taylor Swift'],Lover,We could leave the Christmas lights up 'til Ja...,country


## Drop Unnecessary Index Column

In [3]:
raw_data_df = raw_data_df.drop(raw_data_df.columns[0], axis = 1)

## Clean up `artist` names
Currently, artist names are formatted as ['_name_']. Artist names should be strings without brackets or quoation marks

In [4]:
# Remove preceding [' and ending '] from artist names
raw_data_df["artist"] = raw_data_df["artist"].apply(lambda x: x[2:-2])

# Some songs with multiple artists are now listed as artist_1_', 'artist_2
# Remove ', ' and replace with just ,

# NOTE: This transformation keeps apostrophes in-place
# Where the apostropher is in the artist's name
raw_data_df["artist"] = raw_data_df["artist"].apply(
    lambda x: x.replace("', '", ", ")
)

# Some artist have a randomly inserted backslash in the name
# Clean out the backslash
raw_data_df["artist"] = raw_data_df["artist"].apply(
    lambda x: x.replace("\\", "")
)

## Remove multiple versions of the same songs
Some songs titles are represented twice. This is due to the song having multiple genres. The first step is to consolidate these duplicate titles into one rwo with where the genre value is a list of genres.

In [5]:
# 1. Consolidating songs with two genres
# Count all occurrences of songs in the data
title_counter = Counter(raw_data_df["title"])

# Create a list of only duplicate songs
dupe_songs = []
for title, count in title_counter.items():
    if count > 1:
        dupe_songs.append(title)   

In [6]:
compiled_genre_list = []
for song in dupe_songs:
    # Get a list of singers - in case multiple singers have the same song title
    singer_list = list(raw_data_df.loc[raw_data_df["title"] == song]["artist"].unique())
    # Compile a list of genres for the singer-song combination
    for singer in singer_list:
        sub_df =  raw_data_df.loc[
            (raw_data_df["title"] == song) & (raw_data_df["artist"] == singer)
            ]
        genre_list = list(sub_df["genre"].values)
        lyrics = sub_df.iloc[0,2]
        # Compile information into a dictionary
        new_dict = {
            "artist" : singer, "title" : song,
            "lyrics" : lyrics, "genre": genre_list
            }
        compiled_genre_list.append(new_dict)

In [7]:
# For consistency, put all the genres into a list
raw_genre_df = raw_data_df
raw_genre_df["genre"] = raw_genre_df["genre"].apply(lambda x: [x])

In [8]:
# From the above song Counter
# Create a new dataframe, with songs occuring only once
one_count = [s for s,c in title_counter.items() if c == 1]
one_instance_df = raw_data_df[raw_data_df["title"].isin(one_count)]

# Create dataframe from multi-genre list
multi_genre_df = pd.DataFrame(compiled_genre_list)

# Merge Dataframes together
genre_df = pd.concat([one_instance_df, multi_genre_df]).reset_index()


In [9]:
genre_df.head()

,index,artist,title,lyrics,genre
0,5,Taylor Swift,​betty,"Betty, I won't make assumptions\nAbout why you...",[country]
1,10,John Denver,"Take Me Home, Country Roads","Almost Heaven, West Virginia\nBlue Ridge Mount...",[country]
2,18,Post Malone,Feeling Whitney,"I've been looking for someone...\nOoh, ooh, oo...",[country]
3,20,Cam,Burning House,\n[Verse 1]\nI had a dream about a burning hou...,[country]
4,21,Johnny Cash,Folsom Prison Blues,"I hear the train a-comin', it's rolling 'round...",[country]


## Lyric Preprocessing
Preprocessing Steps:
- Lowercasing
- Tokenize
- Remove text in brackets
- Remove curse words
- Remove all non-apostrophe puncuation - will allow us to remove any stopwords at the end of sentences
- Remove stopwords
- Remove rest of puncuation

### Preprocessing Functions

In [10]:
# Set the puncuation - keep asterik in both sets to maintain any censored profanity
punctuation = set(punctuation) # speeds up comparison
punct_no_apo = punctuation - {"'", "*"}
punct_prof = punctuation - {"*"}

# Stopwords
sw = set(stopwords.words("english"))
# Removing words that were common in our word clouds
stop_add = ['like', 'yeah', 'oh', 'i\'m', 'i\'ve', 'i\'ll', 'can\'t', 'cause']
sw.update(stop_add)

# Function to remove stopwords from TOKENIZED text
def remove_stop(tokens, sw=sw) :
    # modify this function to remove stopwords
    return [token for token in tokens if token.lower() not in sw]

# Function to remove all non-apostrophe from TOKENIZED text
def remove_punctuation_no_apo(tokens, punct_set=punct_no_apo):
    return ["".join(ch for ch in word if ch not in punct_set) for word in tokens]

# Remove all puncuation
def remove_punctuation(tokens, punct_set=punctuation):
    return ["".join(ch for ch in word if ch not in punct_set) for word in tokens]

# Function that will tokenize and strip any whitespace from string
def tokenize(text):
    return text.strip().split()

# Function to remove any singer info (chorus, verse, etc.) located in brackets 
# Text must be TOKENIZED

def remove_bracket(tokens):
    return [token for token in tokens if "[" not in token]

# Pipeline preparation function
def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


### Assemble and Run Pipeline

In [11]:
cleaning_pipeline = [str.lower, profanity.censor, tokenize, remove_bracket, remove_punctuation_no_apo,
                     remove_stop, remove_punctuation]

genre_df['tokens'] = genre_df['lyrics'].apply(prepare, pipeline = cleaning_pipeline)
genre_df['lyrics_clean'] = genre_df['tokens'].apply(lambda l: " ".join(l))
genre_df.head()

,index,artist,title,lyrics,genre,tokens,lyrics_clean
0,5,Taylor Swift,​betty,"Betty, I won't make assumptions\nAbout why you...",[country],"[betty, make, assumptions, switched, homeroom,...",betty make assumptions switched homeroom think...
1,10,John Denver,"Take Me Home, Country Roads","Almost Heaven, West Virginia\nBlue Ridge Mount...",[country],"[almost, heaven, west, virginia, blue, ridge, ...",almost heaven west virginia blue ridge mountai...
2,18,Post Malone,Feeling Whitney,"I've been looking for someone...\nOoh, ooh, oo...",[country],"[looking, someone, ooh, ooh, ooh, ooh, ooh, oo...",looking someone ooh ooh ooh ooh ooh oohooh ooh...
3,20,Cam,Burning House,\n[Verse 1]\nI had a dream about a burning hou...,[country],"[1, dream, burning, house, stuck, inside, get,...",1 dream burning house stuck inside get laid be...
4,21,Johnny Cash,Folsom Prison Blues,"I hear the train a-comin', it's rolling 'round...",[country],"[hear, train, acomin, rolling, round, bend, ai...",hear train acomin rolling round bend aint seen...


### Readout the DF

In [12]:
genre_df.to_csv('data/genre_prepped.csv.gz', compression='gzip')